In [2]:
import re
from itertools import groupby
from datetime import datetime

In [3]:
def returnValue(val):
    return val.group(1) if  val else None

In [4]:
def isPredictMsg(msg):

    patterns = [
        r"Symbol: (.+)",
        r"Position: (.+)",
        r"Leverage: (.+)",
        r"Market: (.+)",
        r"StopLoss: (.+)"
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [5]:
msg =  {
    "id": 8445,
    "date": "2023-12-01T15:25:59+00:00",
    "reply_to_msg_id": None,
    "message": "Symbol: #RNDR/USDT\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 5x\n\nEntry Targets: \n1) 3.5338\n2) 3.6311\n\nTake-Profit Targets: \n1)3.4832\n2) 3.4181\n3) 3.3382\n4) 3.2297\n\nStopLoss: 3.6711\nB.z",
    "edit_date": None,
    "media": None
  }

string = msg['message']
# print(string)
symbol_match = re.search(r"Symbol: (.+)", string)
position_match = re.search(r"Position: (.+)", string)
leverage_match = re.search(r"Leverage: (.+)", string)
market_match = re.search(r"Market: (.+)", string)
stopLoss_match = re.search(r"StopLoss: (.+)", string)

# Extracting values from entry targets
entry_targets_match = re.search(r"Entry Targets:(.+?)\n\n", string, re.DOTALL)
entry_values = re.findall(r"\d+\.\d+", entry_targets_match.group(1)) if entry_targets_match else None

take_profit_targets_match = re.search(r"Take-Profit Targets:(.+?)\n\n", string, re.DOTALL)
profit_values = re.findall(r"\d+\.\d+", take_profit_targets_match.group(1)) if take_profit_targets_match else None


# Creating a dictionary
data = {
    "Symbol": returnValue(symbol_match),
    "Position": returnValue(position_match),
    "Market": returnValue(market_match),
    "Leverage": returnValue(leverage_match),
    "StopLoss": returnValue(stopLoss_match),
    "Entry Targets": {f"Target {i+1}": value for i, value in enumerate(entry_values)} if entry_values else None,
    "Take-Profit Targets": {f"Target {i+1}": value for i, value in enumerate(profit_values)} if profit_values else None,
}

print(data)

{'Symbol': '#RNDR/USDT', 'Position': 'SHORT', 'Market': 'FUTURES', 'Leverage': 'Isolated 5x', 'StopLoss': '3.6711', 'Entry Targets': {'Target 1': '3.5338', 'Target 2': '3.6311'}, 'Take-Profit Targets': {'Target 1': '3.4832', 'Target 2': '3.4181', 'Target 3': '3.3382', 'Target 4': '3.2297'}}


In [6]:
def isEntry(msg, value, symbol):
    entry_price = returnValue(re.search(r"Entry Price: (.+)", msg))
    # for control "average entry"
    if entry_price:
        entry_price = float(re.findall(r"\d+\.\d+", entry_price)[0])
        bigger_number = max(entry_price, float(value))
        smaller_number = min(entry_price, float(value))

        error = 100 * (1 - (smaller_number / bigger_number)) > 1
        if error:
            return False
    else:
        return False

    patterns = [
        r"Entry(.+)",
        rf"{symbol}",

    ]

    # Check if all patterns have a value

    return all(re.search(pattern, msg) for pattern in patterns)

In [7]:
def isTakeProfit(msg, symbol, index):
   

    patterns = [
        r"Take-Profit(.+)",
        r"Profit(.+)",
        rf"target {index}",
        rf"{symbol}",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)

In [8]:
def subtractTime(date1,date2):
    
    # Convert the datetime strings to datetime objects
    datetime1 = datetime.fromisoformat(date1)
    datetime2 = datetime.fromisoformat(date2)

    # Calculate the difference between the datetime objects
    time_difference = datetime2 - datetime1

    # Extract the hours and minutes from the time difference
    hours = time_difference.seconds // 3600
    minutes = (time_difference.seconds // 60) % 60

    # Format the result
    result = f"{hours} Hours {minutes} Minutes"
    return result

In [9]:
predict_messages = [
    {
        "id": 8440,
        "date": "2023-12-01T12:43:33+00:00",
        "reply_to_msg_id": None,
        "message": "Symbol: #GMX/USDT\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 12x\n\nEntry Targets: \n1) 51.07\n2) 51.41\n\nTake-Profit Targets: \n1) 50.80\n2) 50.51\n3) 50.25\n4) 49.80\n\nStopLoss: 51.70\nB.z",
        "edit_date": None,
        "is_predict_msg": True,
        "predict": {
            "Symbol": "#GMX/USDT",
            "Position": "SHORT",
            "Market": "FUTURES",
            "Leverage": "Isolated 12x",
            "StopLoss": "51.70",
            "Entry Targets": [
                {
                    "index": 0,
                    "value": "51.07",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 1,
                    "value": "51.41",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
            ],
            "Take-Profit Targets": [
                {
                    "index": 0,
                    "value": "50.80",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 1,
                    "value": "50.51",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 2,
                    "value": "50.25",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
                {
                    "index": 3,
                    "value": "49.80",
                    "active": False,
                    "Period": None,
                    "date": None,
                },
            ],
            "status": "pending",
        },
        "media": None,
    }
]

grouped_data = {
    "8440": [
        {
            "id": 8446,
            "date": "2023-12-01T15:28:15+00:00",
            "reply_to_msg_id": 8440,
            "message": "Binance Futures\n#GMX/USDT Take-Profit target 2 ✅\nProfit: 9.7513% 📈\nPeriod: 2 Hours 44 Minutes ⏰",
            "edit_date": None,
            "is_predict_msg": False,
            "predict": None,
            "media": None,
        },
        {
            "id": 8444,
            "date": "2023-12-01T14:09:14+00:00",
            "reply_to_msg_id": 8440,
            "message": "Binance Futures, ByBit USDT, KuCoin Futures, OKX Futures\n#GMX/USDT Take-Profit target 1 ✅\nProfit: 6.3442% 📈\nPeriod: 5 Hours 4 Minutes ⏰",
            "edit_date": "2023-12-01T17:47:48+00:00",
            "is_predict_msg": False,
            "predict": None,
            "media": None,
        },
        {
            "id": 8443,
            "date": "2023-12-01T13:51:23+00:00",
            "reply_to_msg_id": 8440,
            "message": "Binance Futures, ByBit USDT, KuCoin Futures, OKX Futures\n#GMX/USDT Entry 1 ✅\nAverage Entry Price: 51.07 💵",
            "edit_date": "2023-12-01T16:36:00+00:00",
            "is_predict_msg": False,
            "predict": None,
            "media": None,
        },
    ],
}


for msg in predict_messages:
    entries = msg["predict"]["Entry Targets"]
    take_profits = msg["predict"]["Take-Profit Targets"]
    symbol = msg["predict"]["Symbol"]

    all_predicts = grouped_data[f'{msg["id"]}']
    isEntryActive = False
    for item in entries:
        for i, value in enumerate(all_predicts):
            if isEntry(value["message"], item["value"], symbol):
                item["active"] = True
                item["date"] = value["date"]
                item["Period"] = subtractTime(msg["date"], value["date"])
                del all_predicts[i]
                isEntryActive = True
                break

    # there is no take-profit if none of entry points is active
    if isEntryActive:
        for j, item in enumerate(take_profits):
            for i, value in enumerate(all_predicts):
                if isTakeProfit(value["message"], symbol , j+1):
                    item["active"] = True
                    item["date"] = value["date"]
                    item["Period"] = returnValue(
                        re.search(r"Period: (.+)", value["message"])
                    )
                    del all_predicts[i]
                    break

predict_messages[0]


{'id': 8440,
 'date': '2023-12-01T12:43:33+00:00',
 'reply_to_msg_id': None,
 'message': 'Symbol: #GMX/USDT\nMarket: FUTURES\nPosition: SHORT\nLeverage: Isolated 12x\n\nEntry Targets: \n1) 51.07\n2) 51.41\n\nTake-Profit Targets: \n1) 50.80\n2) 50.51\n3) 50.25\n4) 49.80\n\nStopLoss: 51.70\nB.z',
 'edit_date': None,
 'is_predict_msg': True,
 'predict': {'Symbol': '#GMX/USDT',
  'Position': 'SHORT',
  'Market': 'FUTURES',
  'Leverage': 'Isolated 12x',
  'StopLoss': '51.70',
  'Entry Targets': [{'index': 0,
    'value': '51.07',
    'active': True,
    'Period': '1 Hours 7 Minutes',
    'date': '2023-12-01T13:51:23+00:00'},
   {'index': 1,
    'value': '51.41',
    'active': False,
    'Period': None,
    'date': None}],
  'Take-Profit Targets': [{'index': 0,
    'value': '50.80',
    'active': True,
    'Period': '5 Hours 4 Minutes ⏰',
    'date': '2023-12-01T14:09:14+00:00'},
   {'index': 1,
    'value': '50.51',
    'active': True,
    'Period': '2 Hours 44 Minutes ⏰',
    'date': '2023

In [10]:
string = "2x-3x-5x"
number = re.findall(r'\d+', string)[0]

print(number)

2
